In [ ]:
import os
from PIL import Image
import numpy as np
import random
from tqdm import tqdm

In [1]:
def read_image_and_crop(image_path, n, crop_size=(100, 100), mean_threshold=0, max_attempts_per_crop=10):
# crop images with a defined maximum number of tries to avoid infinite loops
# crops must exceed a mean threshold

  cropped_images = []
  try:
    img = Image.open(image_path)
    img_width, img_height = img.size

    if img_width < crop_size[0] or img_height < crop_size[1]:
      print(f"Error: Image size ({img_width}x{img_height}) is smaller than crop size ({crop_size[0]}x{crop_size[1]}). Cannot perform cropping.")
      return []

    crops_generated = 0
    while crops_generated < n:
      attempts = 0
      crop_successful = False
      while attempts < max_attempts_per_crop:
        max_x = img_width - crop_size[0]
        max_y = img_height - crop_size[1]

        if max_x < 0 or max_y < 0:
            print("Error: Cannot calculate valid crop area.")
            return []

        x = random.randint(0, max_x)
        y = random.randint(0, max_y)

        bbox = (x, y, x + crop_size[0], y + crop_size[1])

        cropped_img = img.crop(bbox)

        cropped_img_np = np.array(cropped_img)
        mean_val = np.mean(cropped_img_np)

        if mean_val >= mean_threshold:
          cropped_images.append(cropped_img)
          crops_generated += 1
          crop_successful = True
          break

        attempts += 1

      if not crop_successful:
        print(f"Warning: Could not generate a crop with mean >= {mean_threshold} after {max_attempts_per_crop} attempts for crop number {crops_generated + 1}. Skipping this crop slot.")
        crops_generated += 1 # Increment crops_generated to ensure the outer loop terminates


  except FileNotFoundError:
    print(f"Error: Image file not found at {image_path}")
    return []
  except Exception as e:
    print(f"An error occurred while processing the image: {e}")
    return []

  return cropped_images

def process_and_save_crops_from_folder(input_folder, output_base_folder, num_crops_per_image, crop_size=(100, 100), mean_threshold=0, max_attempts_per_crop=10):

    random.seed(42)
    if not os.path.exists(input_folder):
        print(f"Error: Input folder not found at {input_folder}")
        return

    print(f"Processing images from: {input_folder}")

    image_files = [f for f in os.listdir(input_folder) if os.path.isfile(os.path.join(input_folder, f)) and f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')) and f != '.DS_Store' and not f.startswith('._')]

    if not image_files:
        print(f"No image files found in {input_folder}")
        return

    print(f"Found {len(image_files)} image files.")

    for filename in tqdm(image_files):
        image_path = os.path.join(input_folder, filename)
        print(f"\nProcessing image: {filename}")

        cropped_images = read_image_and_crop(image_path, num_crops_per_image, crop_size, mean_threshold, max_attempts_per_crop)

        if cropped_images:
            name_without_extension = os.path.splitext(filename)[0]
            output_folder_path = output_base_folder

            print(f"Saving {len(cropped_images)} crops to {output_folder_path}")
            for i, crop in enumerate(cropped_images):
                crop_filename = f"{name_without_extension}_crop_{i:03d}.jpg"
                crop_save_path = os.path.join(output_folder_path, crop_filename)
                try:
                    if crop.mode != 'RGB':
                        crop = crop.convert('RGB')
                    crop.save(crop_save_path, quality=95)
                except Exception as e:
                    print(f"Error saving crop {i} from {filename}: {e}")
        else:
            print(f"No crops generated for image: {filename}")

    print("\nFinished processing all images.")

In [3]:
chen_synthetic_folder = '/Volumes/T7/drive_thesis/thesis/img/chen/synthetic images'
chen_real_folder = '/Volumes/T7/drive_thesis/thesis/img/chen/real_images'
ag_synthetic_folder = '/Volumes/T7/drive_thesis/thesis/img/assignment group/synthetic images'
ag_real_folder = '/Volumes/T7/drive_thesis/thesis/img/assignment group/real images'
input_folders = [chen_synthetic_folder, ag_synthetic_folder, chen_real_folder, ag_real_folder]

synthetic_folder = '/Volumes/T7/drive_thesis/thesis/img/cropped/1200/n_3/synthetic'
real_folder = '/Volumes/T7/drive_thesis/thesis/img/cropped/1200/n_3/real'
output_folders = [synthetic_folder, synthetic_folder, real_folder, real_folder]

num_crops_to_generate = 3
crop_dimensions = (1200, 1200)

# after manual inspection set to desired value
mean_value_threshold = 50

max_attempts = 20

for i in range(len(input_folders)):

    input_images_folder = input_folders[i]
    output_base_save_folder = output_folders[i]

    process_and_save_crops_from_folder(
        input_images_folder,
        output_base_save_folder,
        num_crops_to_generate,
        crop_size=crop_dimensions,
        mean_threshold=mean_value_threshold,
        max_attempts_per_crop=max_attempts
    )

Processing images from: /Volumes/T7/drive_thesis/thesis/img/chen/synthetic images
Found 1495 image files.


  1%|          | 10/1495 [00:00<00:26, 56.78it/s]


Processing image: DT_482.jpg
Error: Image size (640x640) is smaller than crop size (1200x1200). Cannot perform cropping.
No crops generated for image: DT_482.jpg

Processing image: DT_472.jpg
Error: Image size (640x640) is smaller than crop size (1200x1200). Cannot perform cropping.
No crops generated for image: DT_472.jpg

Processing image: DT_475.jpg
Error: Image size (640x640) is smaller than crop size (1200x1200). Cannot perform cropping.
No crops generated for image: DT_475.jpg

Processing image: DT_473.jpg
Error: Image size (640x640) is smaller than crop size (1200x1200). Cannot perform cropping.
No crops generated for image: DT_473.jpg

Processing image: DT_474.jpg
Error: Image size (640x640) is smaller than crop size (1200x1200). Cannot perform cropping.
No crops generated for image: DT_474.jpg

Processing image: DT_476.jpg
Error: Image size (640x426) is smaller than crop size (1200x1200). Cannot perform cropping.
No crops generated for image: DT_476.jpg

Processing image: DT_

  1%|          | 16/1495 [00:00<00:28, 51.24it/s]

Error: Image size (640x800) is smaller than crop size (1200x1200). Cannot perform cropping.
No crops generated for image: DT_481.jpg

Processing image: DT_479.jpg
Error: Image size (640x640) is smaller than crop size (1200x1200). Cannot perform cropping.
No crops generated for image: DT_479.jpg

Processing image: DT_484.jpg
Error: Image size (640x640) is smaller than crop size (1200x1200). Cannot perform cropping.
No crops generated for image: DT_484.jpg

Processing image: DT_485.jpg
Error: Image size (640x640) is smaller than crop size (1200x1200). Cannot perform cropping.
No crops generated for image: DT_485.jpg

Processing image: DT_486.jpg
Error: Image size (640x480) is smaller than crop size (1200x1200). Cannot perform cropping.
No crops generated for image: DT_486.jpg

Processing image: DT_487.jpg
Error: Image size (640x480) is smaller than crop size (1200x1200). Cannot perform cropping.
No crops generated for image: DT_487.jpg

Processing image: DT_471.jpg
Error: Image size (640

...

Saving 3 crops to /Volumes/T7/drive_thesis/thesis/img/cropped/1200/n_3/real

Processing image: Kamala Harris (48616146657).jpg
Saving 3 crops to /Volumes/T7/drive_thesis/thesis/img/cropped/1200/n_3/real

Processing image: Kamala Harris (48233853667).jpg


100%|█████████▉| 2267/2269 [03:10<00:00,  6.20it/s]

Saving 3 crops to /Volumes/T7/drive_thesis/thesis/img/cropped/1200/n_3/real

Processing image: Kamala Harris (48390414307).jpg
Saving 3 crops to /Volumes/T7/drive_thesis/thesis/img/cropped/1200/n_3/real

Processing image: Kamala Harris (48616144507).jpg


100%|██████████| 2269/2269 [03:10<00:00, 11.89it/s]

Saving 3 crops to /Volumes/T7/drive_thesis/thesis/img/cropped/1200/n_3/real

Finished processing all images.
